In [ ]:
pip install python-bingx

In [ ]:
import requests
import os
from bingX import BingX

api_key = os.environ["BINGX_API_KEY"]
secret_key = os.environ["BINGX_SECRET_KEY"]


bingx_client = BingX(api_key, secret_key)